In [38]:
import pandas as pd
from math import ceil

df13 = pd.read_csv('../../Final Data/candid_opencity13.csv', index_col=0)
df18 = pd.read_csv('../../Final Data/candid_eci18.csv', index_col=0)
df23 = pd.read_csv('../../Final Data/candid_eci23.csv', index_col=0)
const_num = pd.read_csv('../../Final Data/const_num.csv', index_col=0)

In [39]:
df13 = df13.merge(const_num, on='constituency', how='right')
df13.drop(['age','gender', 'candidate_voteshare_percent', 'total_electors', 'district', 'const_category'],axis='columns',inplace=True)
df13.head(5)

,constituency,name,party,total_const_votes,votes,const_num
0,Nippani,Jolle Shashikala Annasaheb,BJP,152690,81860,1
1,Nippani,Jitendra Subhash Nerle,IND,152690,192,1
2,Nippani,Pathan Niyaz,KJP,152690,691,1
3,Nippani,Kakaso Pandurang Patil,INC,152690,63198,1
4,Nippani,Dr. Chandrakant Bashetti Kurabetti,IND,152690,1545,1


In [40]:
df13['constituency'].nunique()

224

In [41]:
df18 = df18.loc[df18['constituency'] != 'Jayanagar'].copy()

df18['total_const_votes'] = df18['votes'] * 100 / df18['percent_votes']
df18['total_const_votes'] = df18['total_const_votes'].apply(lambda x: ceil(x))
df18 = df18.merge(const_num, on='constituency', how='right')
df18[['votes','total_const_votes']] = df18[['votes','total_const_votes']].astype('Int64')
df18.drop(['age','gender', 'percent_votes', 'constituency_category'],axis='columns',inplace=True)
df18.head(5)

,name,party,votes,constituency,total_const_votes,const_num
0,Jolle Shashikala Annasaheb,BJP,87006,Nippani,172251,1
1,Kakaso Pandurang Patil,INC,78500,Nippani,172251,1
2,Kamat Ramesh Ishwar,BSP,2476,Nippani,172251,1
3,NOTA,NOTA,1447,Nippani,172251,1
4,Rohini Shrimant Dixit,AIMEP,1353,Nippani,172251,1


In [42]:
df18['constituency'].nunique()

224

In [43]:
df23 = df23.merge(const_num, on='constituency', how='right')
df23.drop(['age','gender', 'constituency_category', 'total_electors', 'const_num_x'],axis='columns',inplace=True)
df23.rename(columns={'const_num_y':'const_num'}, inplace=True)
df23.head(5)

,constituency,name,party,votes,total_const_votes,const_num
0,Nippani,Jolle Shashikala Annasaheb,BJP,73348.0,187412.0,1
1,Nippani,Uttam Raosaheb Patil,NCP,66056.0,187412.0,1
2,Nippani,Kakasaheb P. Patil,INC,44107.0,187412.0,1
3,Nippani,NOTA,NOTA,919.0,187412.0,1
4,Nippani,Shakuntala Ashok Teli,KRPP,646.0,187412.0,1


In [44]:
df23['constituency'].str.contains('(SC)', regex=False).sum()

np.int64(0)

In [45]:
def process_dataframe(df, year):
    # Add a 'Year' column to identify the year
    df['Year'] = year

    # Sort the dataframe by constituency and votes to identify top 2 candidates in each constituency
    df_sorted = df.sort_values(by=['const_num', 'votes'], ascending=[True, False])

    # Filter top 2 candidates per constituency
    top2 = df_sorted.groupby('const_num').head(2)

    # Create separate winner and runner-up dataframes
    winner = top2.groupby('const_num').nth(0).reset_index()
    runner_up = top2.groupby('const_num').nth(1).reset_index()

    # Rename relevant columns for winner and runner-up
    winner = winner[['const_num', 'constituency', 'name', 'party', 'votes', 'total_const_votes']].add_prefix('winner_')
    runner_up = runner_up[['const_num', 'constituency', 'name', 'party', 'votes']].add_prefix('runner_')

    # Merge winner and runner-up dataframes on constituency number
    merged = pd.merge(winner, runner_up, left_on='winner_const_num', right_on='runner_const_num')

    # Add the year and calculate vote margins
    merged['Year'] = year
    merged['vote_difference'] = merged['winner_votes'] - merged['runner_votes']
    merged['margin_percent'] = (merged['vote_difference'] / merged['winner_total_const_votes']) * 100

    # Retain only necessary columns
    final_columns = [
        'Year', 'winner_const_num', 'winner_constituency', 'winner_total_const_votes',
        'winner_name', 'winner_party', 'winner_votes',
        'runner_name', 'runner_party', 'runner_votes', 'vote_difference', 'margin_percent'
    ]
    merged = merged[final_columns]

    # Rename columns to remove redundancy in the constituency field
    merged.rename(columns={
        'winner_const_num': 'const_num',
        'winner_constituency': 'constituency',
        'winner_total_const_votes': 'total_const_votes'
    }, inplace=True)

    return merged

# Process each dataframe
df13_processed = process_dataframe(df13, 2013)
df18_processed = process_dataframe(df18, 2018)
df23_processed = process_dataframe(df23, 2023)

# Concatenate all processed dataframes
final_df = pd.concat([df13_processed, df18_processed, df23_processed], ignore_index=True)

# Display the combined dataframe
final_df


,Year,const_num,constituency,total_const_votes,winner_name,winner_party,winner_votes,runner_name,runner_party,runner_votes,vote_difference,margin_percent
0,2013,1,Nippani,152690.0,Jolle Shashikala Annasaheb,BJP,81860.0,Kakaso Pandurang Patil,INC,63198.0,18662.0,12.222149
1,2013,2,Chikkodi-Sadalga,149375.0,Prakash Babanna Hukkeri,INC,102237.0,Basavanni Rudrappa Sangappagol,BJP,25649.0,76588.0,51.272301
2,2013,3,Athani,146290.0,Laxman Sangappa Savadi,BJP,74299.0,Mahesh Iranagouda Kumathalli,INC,50528.0,23771.0,16.249231
3,2013,4,Kagwad,129200.0,Bharamgoud Alagoud Kage,BJP,41784.0,Shrimant Balasaheb Patil,JD(S),38897.0,2887.0,2.23452
4,2013,5,Kudachi,115375.0,P.Rajeev,BSRC,71057.0,Ghatage Shama Bhima,INC,24823.0,46234.0,40.072806
...,...,...,...,...,...,...,...,...,...,...,...,...
606,2023,218,Narasimharaja,184941.0,Tanveer Sait,INC,83480.0,S. Satheesh Sandesh Swamy,BJP,52360.0,31120.0,16.826988
607,2023,219,Varuna,199379.0,Siddaramaiah,INC,119816.0,V. Somanna,BJP,73653.0,46163.0,23.153391
608,2023,221,Hanur,180366.0,M.r. Manjunath,JD(S),75632.0,R. Narendra,INC,57978.0,17654.0,9.787876
609,2023,223,Chamarajanagar,173057.0,C. Puttarangashetty,INC,83858.0,V.somanna,BJP,76325.0,7533.0,4.352901


In [46]:
final_df.to_csv('../../Final Data/votemargins.csv', index=True)

In [48]:
final_df.loc[final_df['constituency'] == 'Hagaribommanahalli']

,Year,const_num,constituency,total_const_votes,winner_name,winner_party,winner_votes,runner_name,runner_party,runner_votes,vote_difference,margin_percent
88,2013,89,Hagaribommanahalli,144826.0,Bheemanaik Lbp,JD(S),51972.0,K Nemaraja Naik,BJP,51847.0,125.0,0.08631
312,2018,89,Hagaribommanahalli,176196.0,Bheema Naik L.B.P.,INC,78337.0,K.Nemiraja Naik,BJP,71105.0,7232.0,4.10452


In [49]:
df23.loc[df23['constituency'] == 'Hagaribommanahalli']

,constituency,name,party,votes,total_const_votes,const_num,Year
809,Hagaribommanahalli,NaN,NaN,NaN,NaN,89,2023
